# Bloco jupyter para treinamento e compilação de modelo usando framework aXeleRate


---

Emerson Klippel - 15/07/2020




Baseado no código e tutorial desenvolvidos por Dmitry Maslov https://github.com/AIWintermuteAI/aXeleRate - 



Criando a conexão com o Google Drive e montando a drive na máquina virtual do Google Colab.
Nesse ponto é necessário conectar o bloco de notas ao processamento remoto e selecionar o uso de GPU

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Importando e instalado o framework aXeleRate no VM do Google Colab

In [ ]:
%tensorflow_version 1.x
#we need imgaug 0.4 for image augmentations to work properly, see https://stackoverflow.com/questions/62580797/in-colab-doing-image-data-augmentation-with-imgaug-is-not-working-as-intended
!pip uninstall -y imgaug && pip uninstall -y albumentations && pip install imgaug==0.4
!git clone https://github.com/AIWintermuteAI/aXeleRate.git
import sys
sys.path.append('/content/aXeleRate')
from axelerate import setup_training,setup_inference

No próximo passo o dicionário com as configurações do treinamento são definidos

In [ ]:
config = {
        "model" : {
            "type":                 "Classifier",
            "architecture":         "MobileNet7_5",
            "input_size":           224,
            "fully-connected":      [100,50],
            "labels":               [],
            "dropout" : 		0.5
        },
        "weights" : {
            "full":   				"",
            "backend":   		    "imagenet",
            "save_bottleneck":      False
        
        },
        "train" : {
            "actual_epoch":         10,
            "train_image_folder":   "drive/My Drive/Modelo_Teste/Treinamento",
            "train_times":          4,
            "valid_image_folder":   "drive/My Drive/Modelo_Teste/Validacao",
            "valid_times":          4,
            "valid_metric":         "val_accuracy",
            "batch_size":           4,
            "learning_rate":        1e-3,
            "saved_folder":   		"classifier",
            "first_trainable_layer": "",
            "augumentation":				True
        },
        "converter" : {
            "type":   				["k210","tflite"]
        }
    }

Treinamento e compilação do modelo a partir das configurações do dicionário definido anteriormente

In [ ]:
model_path = setup_training(config_dict=config)

Verificação do modelo a partir do .h5

In [ ]:
from keras import backend as K 
K.clear_session()
setup_inference(config, model_path)